# 建立文字處理的基礎工具

In [ ]:
# 資料庫
group_name = "getNewsData.js"  # 啟動程式
db_path = r'..\ford-news-source.db'  # 資料庫路徑

In [ ]:
from datetime import datetime, timezone

In [ ]:
# 國際時間
def get_utc_time():
    utc_time = datetime.now(timezone.utc)
    return utc_time.strftime("%Y%m%d%H%M%S")

# 客製化String
def trim_string(string):
    return string.strip()


## 資料庫與資料

In [4]:
import sqlite3

### 資料庫- 針對Ford news keyword做蒐集用

In [7]:

# 連接到 SQLite 資料庫
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 建立資料表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS DATA_KEYWORDS (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        KEYWORD TEXT,
        TYPE TEXT,
        VECTORS_BART_ZH BLOB,
        NOTE TEXT,
        DATE INTEGER
    )
''')

conn.commit()

print('完成 DATA_KEYWORDS 資料表建置')

conn.close()

完成 DATA_KEYWORDS 資料表建置


### 資料庫-現有資料讀取

In [ ]:

# 連資料庫
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 從原始資料庫中擷取關鍵字
original_keywords =[]

# 建立資料表
cursor.execute(''' SELECT NEWS_KEYWORDS FROM DATA_NEWS WHERE  COALESCE(NEWS_KEYWORDS, '') !='' ''')

original_keywords = cursor.fetchall()

print(f'取出 {len(original_keywords)} 筆')
print(f'取出 {original_keywords} ')

conn.commit()
conn.close()

In [ ]:
%pip install torch --upgrade
%pip install spacy --upgrade
%pip install transformers --upgrade
%pip install spacy-transformers --upgrade
%python -m spacy download zh_core_web_sm


In [ ]:
%pip install transformers --upgrade

In [ ]:
%pip install -u huggingface_hub
%pip show huggingface-hub



In [ ]:
# 登入
# https://huggingface.co/settings/tokens
!huggingface-cli login --token　已遮蔽

In [ ]:
# 確認是否登入
!huggingface-cli whoami


In [ ]:
# 確認版本
import transformers
print(transformers.__version__)


In [29]:
import spacy
import sqlite3
import torch
from transformers import AutoTokenizer, AutoModel

bert_case = "bert-base-chinese"

def load_spacy_model():
    # 檢查模型是否可用
    nlp = spacy.load('zh_core_web_sm')
    if nlp is not None:
        print("SpaCy 模型載入成功")
    else:
        print("SpaCy 模型載入失敗")

    return nlp

def load_tokenizer_model():
    return AutoTokenizer.from_pretrained(bert_case)


def load_bert_model():
    return AutoModel.from_pretrained(bert_case)


def trim_strings(words):
    # 清除前後空白與空值    
    trimmed_words = [word.strip() for word in words if word.strip()]
    return trimmed_words

def remove_duplicates(lst):
    return list(set(lst))

def insert_new_keywords_to_db(conn, nlp, keywords):
    print('----- insert_new_keywords_to_db ----- S')
    for keyword in keywords:
        doc = nlp(keyword)
        if len(doc) > 0:
            keyword_type = doc[0].pos_
        else:
            keyword_type = "Unknown"

        cursor = conn.cursor()
        cursor.execute('INSERT INTO DATA_KEYWORDS (KEYWORD, TYPE, DATE) VALUES (?, ?, ?)',
                       (keyword, keyword_type, get_utc_time()))
        print(f'資料庫新增：{keyword} ')

    print('----- insert_new_keywords_to_db ----- E')


def update_words_embeddings_to_db(conn, tokenizer, model, words):

    print('----- update_words_embeddings_to_db ----- S')
    for keyword in words:
        # 將句子轉換為模型所需的輸入張量
        input_ids = tokenizer.encode(
            keyword, padding=True, truncation=True, return_tensors="pt", max_length=512)
        inputs = torch.tensor(input_ids)

        # 獲取該句子的嵌入向量
        with torch.no_grad():
            embedding = model(inputs)[0][:, 0, :]  # 取得 [CLS] token 的嵌入向量
            embedding_bytes = embedding.detach().numpy().tobytes()

        # 檢查資料庫中是否已存在相同的句子
        cursor = conn.execute(
            "SELECT id FROM DATA_KEYWORDS WHERE KEYWORD = ?", (keyword,))
        row = cursor.fetchone()
        if row is not None:
            # 若已存在，使用 UPDATE 指令進行更新
            conn.execute(
                "UPDATE DATA_KEYWORDS SET VECTORS_BART_ZH = ? WHERE KEYWORD = ?", (embedding_bytes, keyword))
            print(f'更新：{keyword}')
        else:
            # 若不存在，使用 INSERT 指令進行插入
            conn.execute(
                "INSERT INTO DATA_KEYWORDS (VECTORS_BART_ZH, KEYWORD) VALUES (?, ?)", (embedding_bytes, keyword))
            print(f'新增：{keyword}')

    print('----- update_words_embeddings_to_db ----- E')


def extract_unique_words(data):
    print('----- extract_unique_words ----- S')
    # 將SQL撈出的,分隔字串合併再一起
    # 例如
    # 取出： [('順德,Tesla,電動,充電站,產品,快充',), ('美國,電動,OPTIMAL,車王電,電動巴士',), ('聯嘉,營收',)]
    # 改成: ['充電站', '電動巴士', '車王電', '營收', '快充', '聯嘉', '順德', '產品', 'OPTIMAL', 'Tesla', '電動', '美國']
    words = []
    for row in data:
        keyword_str = row[0]
        keywords = keyword_str.split(',')
        words.extend(keywords)

    clean_words = trim_strings(words)
    unique_words = remove_duplicates(clean_words)

    print(f'整理後資料 {len(unique_words)} 筆')
    print('----- extract_unique_words ----- E')
    return unique_words


def get_news_original_keywords(conn):
    print('----- get_news_original_keywords ----- S')
    # TODO: 未來要增加處理條件
    # 從原始的新聞資料庫中擷取關鍵字，新聞資料的關鍵字都是尚未處理項目，每個array的一塊可能是數個資料串起來
    original_keywords = []

    # 建立資料表
    cursor = conn.cursor()
    cursor.execute(
        ''' SELECT NEWS_KEYWORDS FROM DATA_NEWS WHERE  COALESCE(NEWS_KEYWORDS, '') !='' ''')

    original_keywords = cursor.fetchall()

    print(f'取出 {len(original_keywords)} 筆')
    print('----- get_news_original_keywords ----- E')
    return original_keywords


def exclude_existing_data(conn, words):
    print('----- exclude_existing_data ----- S')
    # 篩選掉已經存在於資料庫中的字
    # TODO: 以後可以讓SQL是動態的
    existing_words = []
    cursor = conn.cursor()
    cursor.execute('SELECT KEYWORD FROM DATA_KEYWORDS')
    for row in cursor.fetchall():
        existing_words.append(row[0])

    filtered_words = [word for word in words if word not in existing_words]
    print(f'預期輸入{len(words)}筆，既有資料庫{len(existing_words)}，篩選後 {len(filtered_words)} 筆')
    print('----- exclude_existing_data ----- E')
    return filtered_words



def process_and_update_keywords(db_path):
    conn = None
    try:
        # 連接到 SQLite 資料庫
        conn = sqlite3.connect(db_path)

        # 載入 SpaCy 模型
        nlp = load_spacy_model()

        # 載入 tokenizer 和 BERT 模型
        tokenizer = load_tokenizer_model()
        model = load_bert_model()

        # 取得資料
        news_original_keywords = get_news_original_keywords(conn)
        unique_keywords = extract_unique_words(news_original_keywords)

        # 拆分並整理關鍵字
        # processed_keywords = process_keywords(original_keywords)
        print(f'DATA_NEWS 目前有 {len(unique_keywords)} 筆關鍵字')

        # 篩選掉已經存在於資料庫中的關鍵字 (注意，後面處理資料以Array為核心)
        new_keywords = exclude_existing_data(conn, unique_keywords)
        print(f'排除現有DATA_KEYWORDS，預計新增 {len(new_keywords)} 筆關鍵字')

        # 將整理後的關鍵字儲存到資料庫中
        insert_new_keywords_to_db(conn, nlp, new_keywords)

        # 更新嵌入向量到資料庫中
        update_words_embeddings_to_db(conn, tokenizer, model, new_keywords)

        # 儲存變更
        conn.commit()
    except Exception as e:
        print(f"發生錯誤：{str(e)}")
        print(f"一切儲存已經終止")
    finally:
        if conn is not None:
            conn.close()


# 使用範例
process_and_update_keywords(db_path)

SpaCy 模型載入成功


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


----- get_news_original_keywords ----- S
取出 751 筆
----- get_news_original_keywords ----- E
----- extract_unique_words ----- S
整理後資料 1278 筆
----- extract_unique_words ----- E
DATA_NEWS 目前有 1278 筆關鍵字
----- exclude_existing_data ----- S
預期輸入1278筆，既有資料庫0，篩選後 1278 筆
----- exclude_existing_data ----- E
排除現有DATA_KEYWORDS，預計新增 1278 筆關鍵字
----- insert_new_keywords_to_db ----- S
資料庫新增：Lightning 
資料庫新增：市占 
資料庫新增：本田 
資料庫新增：社群 
資料庫新增：德國 
資料庫新增：type:post 
資料庫新增：訪中 
資料庫新增：現代汽車 
資料庫新增：car 
資料庫新增：英國復古跑車 
資料庫新增：日本 
資料庫新增：菜單 
資料庫新增：7人座 
資料庫新增：寶馬 
資料庫新增：電動馬達 
資料庫新增：J1772 
資料庫新增：BuyCarTv 
資料庫新增：紅杉 
資料庫新增：市佔率 
資料庫新增：即時新聞 
資料庫新增：嘉泥 
資料庫新增：休旅車 
資料庫新增：密西根州 
資料庫新增：X 
資料庫新增：wagon 
資料庫新增：旅行車 
資料庫新增：一次 
資料庫新增：Crown 
資料庫新增：AC 
資料庫新增：Mitsubishi 歐洲 推出 全新 Colt 台灣 國產化 引進 
資料庫新增：撞擊 
資料庫新增：夏季 
資料庫新增：RTR 
資料庫新增：TRD 
資料庫新增：報稅 
資料庫新增：Chevrolet 推出 最後 一款 第六代 Camaro 紀念版 Panther 
資料庫新增：嘉偉 
資料庫新增：video 
資料庫新增：Formula 1 
資料庫新增：Roadster 
資料庫新增：優惠 
資料庫新增：ARTC 
資料庫新增：體質 
資料庫新增：長期 
資料庫新增：Fisker 
資料庫新增：智慧駕駛 
資料庫新增：Ferrari 
資料庫新增：信義新天地 

C:\Users\mudalse\AppData\Local\Temp\ipykernel_11488\4241739644.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(input_ids)


更新：市占
更新：本田
更新：社群
更新：德國
更新：type:post
更新：訪中
更新：現代汽車
更新：car
更新：英國復古跑車
更新：日本
更新：菜單
更新：7人座
更新：寶馬
更新：電動馬達
更新：J1772
更新：BuyCarTv
更新：紅杉
更新：市佔率
更新：即時新聞
更新：嘉泥
更新：休旅車
更新：密西根州
更新：X
更新：wagon
更新：旅行車
更新：一次
更新：Crown
更新：AC
更新：Mitsubishi 歐洲 推出 全新 Colt 台灣 國產化 引進
更新：撞擊
更新：夏季
更新：RTR
更新：TRD
更新：報稅
更新：Chevrolet 推出 最後 一款 第六代 Camaro 紀念版 Panther
更新：嘉偉
更新：video
更新：Formula 1
更新：Roadster
更新：優惠
更新：ARTC
更新：體質
更新：長期
更新：Fisker
更新：智慧駕駛
更新：Ferrari
更新：信義新天地
更新：台北國際車用電子展
更新：美銀證券
更新：FORD TRANSIT XJ220
更新：車廠
更新：車
更新：做空機構
更新：Kg
更新：評級
更新：Pro
更新：VOLVO
更新：Volkswagen T-Roc
更新：Focus Wagon
更新：6000
更新：品牌
更新：suv
更新：大改款
更新：轉虧為盈
更新：WildtrakX
更新：美規
更新：CCS2
更新：Drag
更新：經濟衰退
更新：重型機車
更新：引擎
更新：CHAdeMO
更新：2024 Ford Mustang RTR Spec 2
更新：半導體
更新：影音系統
更新：2024年
更新：Opel
更新：採購副總
更新：投資人
更新：試車頻道
更新：蝙蝠俠機車
更新：Wards
更新：WRX Wagon
更新：科技類股
更新：Capstone
更新：Kuga
更新：空間
更新：三菱
更新：HR-V
更新：銷售
更新：專題報導
更新：女性墮胎權
更新：cat:新聞
更新：自行車
更新：Ford 正式 發佈 Mustang GT3 賽車 將於 2024 參加  Le Mans  24 小時 耐力賽
更新：電動車市
更新：地方新聞
更新：股價
更新：CX-90
更新：國運
更新：sports
更新：特斯拉 Cybertruck 照片 外洩 內裝 設計 細節 

In [ ]:
import sqlite3
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
import heapq

# 初始化 tokenizer 和模型
bert = "bert-base-chinese"
chinese_tokenizer = AutoTokenizer.from_pretrained(bert)
chinese_model = AutoModel.from_pretrained(bert)

# 查詢句子
query = "外國"


# 將查詢句子轉換為模型所需的輸入張量
query_input = chinese_tokenizer(
    [query], padding=True, truncation=True, return_tensors="pt", max_length=512)
query_embedding = chinese_model(
    **query_input).last_hidden_state[:, 0, :]  # 取得 [CLS] token 的嵌入向量

# 連接到 SQLite 資料庫
connection = sqlite3.connect(db_path)

# 從資料庫中檢索嵌入向量
cursor = connection.execute("SELECT VECTORS_BART_ZH, KEYWORD FROM DATA_KEYWORDS")

# 使用最小堆維護相似度最高的前三個句子
similar_sentences = []
heap_size = 20

# 比較查詢句子和每個 CLS token 嵌入向量的相似度
for row in cursor:
    embedding_bytes = row[0]
    embedding = torch.from_numpy(np.frombuffer(
        embedding_bytes, dtype=np.float32)).unsqueeze(0)
    similarity = torch.nn.functional.cosine_similarity(
        query_embedding, embedding).item()

    # 將句子及相似度加入最小堆
    if len(similar_sentences) < heap_size:
        heapq.heappush(similar_sentences, (similarity, row[1]))
    else:
        heapq.heappushpop(similar_sentences, (similarity, row[1]))

# 關閉連接
connection.close()

# 取得前三個相似的句子及相似度
similar_sentences = sorted(similar_sentences, reverse=True)
top_similarities = [similarity for similarity, _ in similar_sentences]
top_sentences = [sentence for _, sentence in similar_sentences]

print("\n\n查詢結果如下：")
sentence_num = 0.0
is_low_similar = 0
# 列印前三個相似的句子及相似度
for similarity, sentence in zip(top_similarities, top_sentences):
    print("句子：", sentence)
    print("相似度：", similarity)
    print()
    sentence_num += 1
    is_low_similar += similarity

print(f'共{sentence_num}筆。 \n您查詢句子:\n[{query}]。\n')


# 判斷平均相似度是否偏低
similarity_need_more_then = 0.5
if sentence_num > 0 and similarity_need_more_then > (is_low_similar / sentence_num):
    print(
        f"\n注意：您的搜尋句子在資料庫中相似度偏低(平均低於{similarity_need_more_then:.2f})。")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).




查詢結果如下：
句子： 福特
相似度： 1.0

句子： 福斯
相似度： 0.9376312494277954

句子： 特斯拉
相似度： 0.9220630526542664

句子： 福斯汽車
相似度： 0.9206817150115967

句子： 福特汽車
相似度： 0.916707456111908

句子： 通用汽車
相似度： 0.9141786098480225

句子： 雪鐵龍
相似度： 0.9033083319664001

句子： 馬斯克
相似度： 0.9007899761199951

句子： 伯恩斯
相似度： 0.8984288573265076

句子： 法拉利
相似度： 0.8944268226623535

句子： 格斯
相似度： 0.8932355046272278

句子： 福特六和
相似度： 0.8931446671485901

句子： 長安福特
相似度： 0.8888293504714966

句子： FORD 澳洲/福特
相似度： 0.8855004906654358

句子： ai
相似度： 0.8849354982376099

句子： 比爾
相似度： 0.8828038573265076

句子： 瑪莎拉帝
相似度： 0.8806556463241577

句子： 法利
相似度： 0.8795092105865479

句子： car
相似度： 0.8793715238571167

句子： 馬丁
相似度： 0.8790813088417053

共20.0筆。 
您查詢句子:
[福特]。

